# GPU-Accelerated Curve Fitting with Theseus

In [28]:
import numpy as np
from theseusqfls import genplanck
import pandas as pd

In [29]:
energy, pl = genplanck.import_hyperspectral('theseusqfls/data/example_photometric.h5')

In [30]:
result = genplanck.fit_qfls(energy, pl)

ValueError: object too deep for desired array

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\cbc37\Anaconda3\envs\theseus\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\cbc37\AppData\Local\Temp\ipykernel_14664\1141211487.py", line 1, in <module>
    result = genplanck.fit_qfls(energy, pl)
  File "C:\Users\cbc37\Desktop\theseusqfls\theseusqfls\genplanck.py", line 155, in fit_qfls
    pl_tensor = torch.tensor(pl.flatten())
  File "C:\Users\cbc37\Anaconda3\envs\theseus\lib\site-packages\numpy\__init__.py", line 311, in __getattr__
    raise AttributeError("module {!r} has no attribute "
AttributeError: module 'numpy' has no attribute 'flatten'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\cbc37\Anaconda3\envs\theseus\lib\site-packages\IPython\core\interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C

In [31]:
pl.flatten()

array([0.0000000e+00, 1.1357260e+17, 5.6786305e+16, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)